# Bachelorprojekt

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sympy import symbols, solve
import itertools
import yfinance as yf
from sklearn.linear_model import LinearRegression

# Import data

In [2]:
indexes = yf.download("TSLA KO V SHEL", start="2011-02-01", end="2022-02-01")

[*********************100%***********************]  4 of 4 completed


In [3]:
returns = indexes['Adj Close'].resample('y').ffill().pct_change()
returns

,KO,SHEL,TSLA,V
Date,,,,
2011-12-31,NaN,NaN,NaN,NaN
2012-12-31,0.065276,-0.009087,0.185924,0.504669
2013-12-31,0.172330,0.090379,3.441394,0.480564
2014-12-31,0.052661,-0.012650,0.478495,0.186358
2015-12-31,0.051398,-0.271207,0.079133,0.191374
2016-12-31,-0.003571,0.282596,-0.109662,0.013711
2017-12-31,0.143821,0.310229,0.457017,0.471765
2018-12-31,0.067735,-0.075132,0.068894,0.164876
2019-12-31,0.205994,0.077085,0.257001,0.433270


In [4]:
returns = returns[1:]
returns

,KO,SHEL,TSLA,V
Date,,,,
2012-12-31,0.065276,-0.009087,0.185924,0.504669
2013-12-31,0.172330,0.090379,3.441394,0.480564
2014-12-31,0.052661,-0.012650,0.478495,0.186358
2015-12-31,0.051398,-0.271207,0.079133,0.191374
2016-12-31,-0.003571,0.282596,-0.109662,0.013711
2017-12-31,0.143821,0.310229,0.457017,0.471765
2018-12-31,0.067735,-0.075132,0.068894,0.164876
2019-12-31,0.205994,0.077085,0.257001,0.433270
2020-12-31,0.024691,-0.374393,7.434370,0.171242


In [5]:
cov_matrix = returns.cov()

In [6]:
returns.mean()

KO      0.084042
SHEL    0.044219
TSLA    1.152410
V       0.241667
dtype: float64

In [7]:
len(returns.mean())

4

# Import ESG data

In [42]:
ESG_data = pd.read_csv("ESG_DATA.csv") 

In [43]:
ESG_data

,Isin,CurrencyCode,company_name,environment_grade,environment_level,environment_score,governance_grade,governance_level,governance_score,social_grade,social_level,social_score,stock_symbol,total,total_grade,total_level
0,SE0011337708,SEK,AAK AB (publ.),AA,Excellent,647,BB,Medium,345,BB,Medium,307,AAK,1299,A,High
1,CH0012221716,SEK,ABB Ltd,BB,Medium,306,B,Medium,220,BBB,High,425,ABB,951,BBB,High
2,SE0007897079,SEK,ACADIA Pharmaceuticals Inc.,B,Medium,290,BB,Medium,300,BB,Medium,344,ACAD,934,BBB,High
3,SE0014781795,SEK,Addtech AB (publ.),A,High,500,BB,Medium,300,BB,Medium,300,ADDT-B,1100,BBB,High
4,CA00830W1059,SEK,AEC Securities Public Company Limited,B,Medium,240,BB,Medium,300,BB,Medium,300,AEC,840,BB,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,DK0060036564,DKK,Spar Nord Bank A/S,B,Medium,230,BB,Medium,300,B,Medium,216,SPNO,746,B,Medium
283,DK0060696300,DKK,Straker Translations Limited,BBB,High,410,BB,Medium,315,BB,Medium,301,STG,1026,BBB,High
284,DK0010311471,DKK,Sydbank A/S,A,High,500,BB,Medium,300,BB,Medium,301,SYDB,1101,BBB,High
285,DK0060477503,DKK,Thorney Opportunities Ltd,B,Medium,215,BB,Medium,305,B,Medium,203,TOP,723,B,Medium


In [44]:
lol = ESG_data[ESG_data['environment_level']=='Excellent']
lol

,Isin,CurrencyCode,company_name,environment_grade,environment_level,environment_score,governance_grade,governance_level,governance_score,social_grade,social_level,social_score,stock_symbol,total,total_grade,total_level
0,SE0011337708,SEK,AAK AB (publ.),AA,Excellent,647,BB,Medium,345,BB,Medium,307,AAK,1299,A,High
191,NO0010629108,NOK,NextSource Materials Inc.,AA,Excellent,600,BB,Medium,315,BB,Medium,327,NEXT,1242,A,High
252,DK0010263722,DKK,AGF Management Limited,AA,Excellent,700,BB,Medium,350,BB,Medium,373,AGF-B,1423,A,High
272,DK0060094928,DKK,Ørsted A/S,AA,Excellent,693,BB,Medium,340,BB,Medium,347,ORSTED,1380,A,High


In [45]:
symboler = lol['stock_symbol'].to_numpy()
symboler

array(['AAK', 'NEXT', 'AGF-B', 'ORSTED'], dtype=object)

In [46]:
symboler[0]

'AAK'

In [47]:
symboler

array(['AAK', 'NEXT', 'AGF-B', 'ORSTED'], dtype=object)

In [48]:
symboler[:]

array(['AAK', 'NEXT', 'AGF-B', 'ORSTED'], dtype=object)

In [49]:
ESG_data['stock_symbol']

0         AAK
1         ABB
2        ACAD
3      ADDT-B
4         AEC
        ...  
282      SPNO
283       STG
284      SYDB
285       TOP
286      TRYG
Name: stock_symbol, Length: 287, dtype: object

# Efficient frontier

# 100% copy paste fra GPT, ved ikke om det virker

In [1]:
class Frontier:
    
    def __init__(self, cov, perf):
        self.cov = cov
        self.zbar = perf
        
    def vmap2(self, f, v1, v2):
        if len(v1) != len(v2):
            raise IndexError("Vectors have different lengths.")
        else:
            return [f(x, v2[i]) for i, x in enumerate(v1)]
        
    def frontier(self):
        Sinv = np.linalg.inv(self.cov)
        unity = np.ones(len(self.zbar))
        unity = list(map(float, unity))
        Sinv1 = np.dot(Sinv, unity)
        SinvZbar = np.dot(Sinv, self.zbar)
        A = np.dot(unity, Sinv1)
        B = np.dot(unity, SinvZbar)
        C = np.dot(self.zbar, SinvZbar)
        D = A*C-B*B
        
        def plot(muv):
            minvar = [(mu, (A*mu*mu - 2.0*B*mu + C) / D) for mu in muv]
            minstd = [(mu, np.sqrt(m)) for mu, m in minvar]
            return minstd
        
        def weights(mu):
            lambda_ = (C - mu*B) / D
            gamma = (mu*A - B) / D
            Sinv1_ = list(map(float, Sinv1))
            SinvZbar_ = list(map(float, SinvZbar))
            ws = self.vmap2(lambda x, y: lambda_*x + gamma*y, Sinv1_, SinvZbar_)
            return ws
        
        return {'plot': plot, 'weights': weights}


# Eksempel med hvordan kalder koden

In [2]:
# Create a sample covariance matrix and performance vector
cov = np.array([[0.04, 0.006, 0.02], [0.006, 0.09, 0.05], [0.02, 0.05, 0.16]])
perf = np.array([0.05, 0.10, 0.15])

# Create an instance of the Frontier class
f = Frontier(cov, perf)

# Call the frontier method to get a dictionary containing the plot and weights functions
frontier_dict = f.frontier()

# Call the plot function with a list of return levels to get a list of (mu, sigma) tuples representing points on the efficient frontier
points_on_frontier = frontier_dict['plot']([0.05, 0.10, 0.15])

# Call the weights function with a return level to get a list of portfolio weights corresponding to that return level
weights_at_return_level = frontier_dict['weights'](0.10)


NameError: name 'np' is not defined

# Test af load data

In [70]:
#isindata = ESG_data['Isin'].tolist()
ESG_data = pd.read_csv("ESG_DATA.csv") 
isindata = ESG_data[ESG_data['environment_level']=='High']
isindata = isindata['Isin'].tolist()
isindata

['SE0014781795',
 'SE0000695876',
 'SE0010468116',
 'SE0003883990',
 'SE0007666110',
 'GB0009895292',
 'SE0000862997',
 'SE0015811625',
 'JE00BLD8Y945',
 'SE0000418923',
 'SE0007691613',
 'SE0010520106',
 'SE0013747870',
 'SE0012853455',
 'SE0009922156',
 'SE0011166974',
 'CA31730E1016',
 'SE0005468717',
 'SE0001790791',
 'SE0011090000',
 'SE0007074281',
 'SE0008040653',
 'SE0001279142',
 'SE0001515552',
 'SE0000190126',
 'SE0016075063',
 'SE0000936478',
 'SE0012323715',
 'SE0007871645',
 'SE0010100958',
 'SE0014504817',
 'CA5503721063',
 'SE0000108847',
 'MT0000580101',
 'SE0018012486',
 'SE0000375115',
 'SE0000118952',
 'SE0015988019',
 'SE0011528017',
 'SE0000949331',
 'SE0015962477',
 'SE0009806284',
 'SE0000106205',
 'US7170811035',
 'SE0018013849',
 'SE0017232838',
 'SE0000191090',
 'SE0004635878',
 'SE0000148884',
 'SE0000163594',
 'SE0009268279',
 'SE0015346895',
 'SE0000872095',
 'SE0014960365',
 'SE0005190220',
 'SE0000114837',
 'SE0001834821',
 'SE0011205202',
 'SE0000115420

In [71]:
isindata[0][:2]

'SE'

In [72]:
# Permanently changes the pandas settings
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)
 
# All dataframes hereafter reflect these changes.
#display(df)
 
# Resets the options
pd.reset_option('all')

C:\Users\gusta\AppData\Local\Temp\ipykernel_21444\1938921825.py:11: FutureWarning: column_space is deprecated and will be removed in a future version. Use df.to_string(col_space=...) instead.
  pd.reset_option('all')
C:\Users\gusta\AppData\Local\Temp\ipykernel_21444\1938921825.py:11: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.
  pd.reset_option('all')
C:\Users\gusta\AppData\Local\Temp\ipykernel_21444\1938921825.py:11: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  pd.reset_option('all')


In [73]:
isin_dk = []
isin_se = []
isin_no = []
for i in range(0,len(isindata)):
    if isindata[i][:2] == 'DK':
        isin_dk.append(isindata[i])
    if isindata[i][:2] == 'SE':
        isin_se.append(isindata[i])
    if isindata[i][:2] == 'NO':
        isin_no.append(isindata[i])

print(isin_dk)
print(isin_se)
print(isin_no)

['DK0060520450', 'DK0060946788', 'DK0060655629', 'DK0060079531', 'DK0061277894', 'DK0060522746', 'DK0010234467', 'DK0010272202', 'DK0010272632', 'DK0060542181', 'DK0060952919', 'DK0060634707', 'DK0060854669', 'DK0010219070', 'DK0010267129', 'DK0060696300', 'DK0010311471']
['SE0014781795', 'SE0000695876', 'SE0010468116', 'SE0003883990', 'SE0007666110', 'SE0000862997', 'SE0015811625', 'SE0000418923', 'SE0007691613', 'SE0010520106', 'SE0013747870', 'SE0012853455', 'SE0009922156', 'SE0011166974', 'SE0005468717', 'SE0001790791', 'SE0011090000', 'SE0007074281', 'SE0008040653', 'SE0001279142', 'SE0001515552', 'SE0000190126', 'SE0016075063', 'SE0000936478', 'SE0012323715', 'SE0007871645', 'SE0010100958', 'SE0014504817', 'SE0000108847', 'SE0018012486', 'SE0000375115', 'SE0000118952', 'SE0015988019', 'SE0011528017', 'SE0000949331', 'SE0015962477', 'SE0009806284', 'SE0000106205', 'SE0018013849', 'SE0017232838', 'SE0000191090', 'SE0004635878', 'SE0000148884', 'SE0000163594', 'SE0009268279', 'SE001

In [75]:
from pandas_datareader import data as pdr
yf.pdr_override() # <== that's all it takes :-)

# download dataframe
data_se = pdr.get_data_yahoo(isin_se, start="2010-01-01", end="2021-04-30")
data_dk = pdr.get_data_yahoo(isin_dk, start="2010-01-01", end="2021-04-30")
data_no = pdr.get_data_yahoo(isin_no, start="2010-01-01", end="2021-04-30")

[*********************100%***********************]  53 of 53 completed

4 Failed downloads:
- BRILL.ST: Data doesn't exist for startDate = 1262300400, endDate = 1619733600
- PURE.ST: Data doesn't exist for startDate = 1262300400, endDate = 1619733600
- SFL.ST: Data doesn't exist for startDate = 1262300400, endDate = 1619733600
- INT.ST: Data doesn't exist for startDate = 1262300400, endDate = 1619733600
[*********************100%***********************]  17 of 17 completed

1 Failed download:
- : No timezone found, symbol may be delisted
[*********************100%***********************]  53 of 53 completed

2 Failed downloads:
- : No timezone found, symbol may be delisted
- ELO.OL: Data doesn't exist for startDate = 1262300400, endDate = 1619733600


In [76]:
display(data_no['Adj Close'])
display(data_dk['Adj Close'])
display(data_se['Adj Close'])

,NO0003033102,NO0003049405,NO0003053308,NO0003053605,NO0003078800,NO0003079709,NO0003080608,NO0003095309,NO0003097503,NO0003103103,...,NO0010791353,NO0010813843,NO0010816093,NO0010816895,NO0010833262,NO0010840507,NO0010861115,NO0010927288,NO0011002586,NO0012535832
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,10.013632,1.591135,219.468338,31.237755,63.867905,2.354740,84255.750000,20.642979,15.093127,22.200001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,9.995225,1.588509,224.887329,31.068068,63.572220,2.295708,87037.781250,20.249781,15.093127,22.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-06,10.400188,1.575381,219.468338,31.268608,63.867905,2.315385,85448.046875,20.839579,14.338472,21.400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,10.621077,1.575381,216.758865,33.058033,65.287178,2.295708,87832.632812,20.544680,14.338472,21.299999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,11.688707,1.509740,210.797989,33.212296,64.991493,2.341622,84653.187500,21.724277,14.338472,21.799999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-23,2.770000,11.160000,1.980000,75.669472,109.581871,19.865913,6.800000,5.100000,86.557137,6.440000,...,10.640926,1.020,31.937046,12.359623,43.364788,84.199997,37.700001,52.887573,NaN,36.072861
2021-04-26,2.810000,11.040000,2.040000,77.316963,113.249886,19.527567,6.660000,4.970000,86.952377,6.400000,...,11.007854,0.985,32.194012,12.723141,42.776123,88.500000,37.750000,52.887573,NaN,37.898331
2021-04-27,2.772000,11.040000,2.005000,76.780571,110.682274,19.914251,6.690000,4.970000,87.545235,6.400000,...,11.582177,0.990,29.202208,12.586823,43.168568,89.349998,37.200001,52.887573,NaN,39.088852


,DK0010219070,DK0010234467,DK0010267129,DK0010272202,DK0010272632,DK0010311471,DK0060079531,DK0060520450,DK0060522746,DK0060542181,DK0060634707,DK0060655629,DK0060696300,DK0060854669,DK0060946788,DK0060952919,DK0061277894
Date,,,,,,,,,,,,,,,,,
2010-01-04,523.826782,293.015289,5.163384,92.000000,27.892155,84.648483,88.650734,NaN,NaN,NaN,19.708996,46.551464,NaN,98.603470,5.007597,NaN,NaN
2010-01-05,534.433960,289.625641,5.163384,89.599998,27.516500,85.582100,88.604576,NaN,NaN,NaN,19.084412,48.120602,NaN,97.967323,4.961657,NaN,NaN
2010-01-06,514.035828,282.394379,4.896312,88.750000,26.671284,85.706589,92.806259,NaN,NaN,NaN,19.015018,48.512897,NaN,97.490211,5.191363,NaN,NaN
2010-01-07,518.115417,283.223022,4.940824,88.000000,27.704327,85.893303,93.267967,NaN,NaN,NaN,19.362007,48.512897,NaN,98.285393,5.053539,NaN,NaN
2010-01-08,507.508209,283.147675,5.519478,89.000000,27.704327,87.449341,93.914368,NaN,NaN,NaN,19.153814,49.035946,NaN,99.398666,5.099482,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-23,2500.000000,225.237671,179.800003,2251.000000,556.785645,149.667099,1305.774414,18.820000,122.0,121.800003,711.187134,348.485046,109.949928,603.590515,343.215057,653.0,NaN
2021-04-26,2515.000000,234.089813,178.399994,2271.000000,560.765503,152.655167,1287.446045,19.700001,121.0,122.000000,710.419312,357.400665,109.760361,618.360229,348.905365,653.0,NaN
2021-04-27,2465.000000,234.188171,176.000000,2269.000000,563.949463,152.743057,1374.629761,20.000000,121.0,119.250000,709.459595,358.563568,109.476006,610.483093,354.795319,648.0,NaN


,SE0000106205,SE0000108847,SE0000114837,SE0000115420,SE0000118952,SE0000148884,SE0000163594,SE0000190126,SE0000191090,SE0000375115,...,SE0014781795,SE0014960365,SE0015346895,SE0015811625,SE0015962477,SE0015988019,SE0016075063,SE0017232838,SE0018012486,SE0018013849
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,27.137428,78.387642,37.431538,40.340775,35.277679,24.287655,46.695545,62.684044,63.488369,7.654094,...,6.139913,15.064403,NaN,NaN,2.877228,3.614271,NaN,NaN,25.338188,15.991792
2010-01-05,27.194559,78.387642,37.603241,41.408329,35.494995,24.186434,47.185707,63.376675,64.721153,7.944756,...,6.235000,15.064403,NaN,NaN,3.001246,3.729213,NaN,NaN,25.219095,15.776655
2010-01-07,27.194559,77.850723,36.916424,42.120037,35.784748,24.559349,46.858929,63.376675,65.645737,8.138528,...,6.302919,13.773167,NaN,NaN,2.939237,3.793069,NaN,NaN,25.219095,16.063503
2010-01-08,27.137428,78.387642,37.774948,42.249443,35.350117,25.438370,47.120354,63.723011,65.645737,8.380747,...,6.357254,13.773167,NaN,NaN,2.976442,3.831384,NaN,NaN,25.569363,17.856306
2010-01-11,26.794645,77.313835,38.427422,42.831745,34.987919,25.736710,47.381771,63.723011,68.727699,8.913628,...,6.438758,13.988370,NaN,NaN,2.864825,3.831384,NaN,NaN,25.646420,17.354321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-23,109.130219,485.646912,219.865067,199.625015,147.517456,95.918304,138.419556,317.484070,56.695541,239.486389,...,149.288803,147.465515,NaN,NaN,79.953339,78.615593,NaN,NaN,75.494644,13.360000
2021-04-26,111.481010,481.082184,220.255051,198.519058,149.866440,97.113266,138.608383,315.923950,56.695541,243.561920,...,146.342316,150.414825,NaN,NaN,81.094170,79.085609,NaN,NaN,76.041710,13.420000
2021-04-27,110.124786,481.280670,220.742569,199.993668,149.396652,97.572861,137.522552,316.899017,58.853500,247.443390,...,148.306656,146.482422,NaN,NaN,82.615280,79.332985,NaN,NaN,76.041710,13.080000


In [77]:
result_se = []
result_dk = []
result_no = []
for col in data_dk.columns:
    if data_dk[col].isnull().all() == True:
        result_dk.append(col)

for col in data_se.columns:
    if data_se[col].isnull().all() == True:
        result_se.append(col)

for col in data_no.columns:
    if data_no[col].isnull().all() == True:
        result_no.append(col)

In [78]:
data_dk = data_dk.drop(columns=result_dk)
data_se = data_se.drop(columns=result_se)
data_no = data_no.drop(columns=result_no)

In [79]:
display(data_dk['Adj Close'])
display(data_se['Adj Close'])
display(data_no['Adj Close'])

,DK0010219070,DK0010234467,DK0010267129,DK0010272202,DK0010272632,DK0010311471,DK0060079531,DK0060520450,DK0060522746,DK0060542181,DK0060634707,DK0060655629,DK0060696300,DK0060854669,DK0060946788,DK0060952919
Date,,,,,,,,,,,,,,,,
2010-01-04,523.826782,293.015289,5.163384,92.000000,27.892155,84.648483,88.650734,NaN,NaN,NaN,19.708996,46.551464,NaN,98.603470,5.007597,NaN
2010-01-05,534.433960,289.625641,5.163384,89.599998,27.516500,85.582100,88.604576,NaN,NaN,NaN,19.084412,48.120602,NaN,97.967323,4.961657,NaN
2010-01-06,514.035828,282.394379,4.896312,88.750000,26.671284,85.706589,92.806259,NaN,NaN,NaN,19.015018,48.512897,NaN,97.490211,5.191363,NaN
2010-01-07,518.115417,283.223022,4.940824,88.000000,27.704327,85.893303,93.267967,NaN,NaN,NaN,19.362007,48.512897,NaN,98.285393,5.053539,NaN
2010-01-08,507.508209,283.147675,5.519478,89.000000,27.704327,87.449341,93.914368,NaN,NaN,NaN,19.153814,49.035946,NaN,99.398666,5.099482,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-23,2500.000000,225.237671,179.800003,2251.000000,556.785645,149.667099,1305.774414,18.820000,122.0,121.800003,711.187134,348.485046,109.949928,603.590515,343.215057,653.0
2021-04-26,2515.000000,234.089813,178.399994,2271.000000,560.765503,152.655167,1287.446045,19.700001,121.0,122.000000,710.419312,357.400665,109.760361,618.360229,348.905365,653.0
2021-04-27,2465.000000,234.188171,176.000000,2269.000000,563.949463,152.743057,1374.629761,20.000000,121.0,119.250000,709.459595,358.563568,109.476006,610.483093,354.795319,648.0


,SE0000106205,SE0000108847,SE0000114837,SE0000115420,SE0000118952,SE0000148884,SE0000163594,SE0000190126,SE0000191090,SE0000375115,...,SE0012323715,SE0012853455,SE0013747870,SE0014504817,SE0014781795,SE0014960365,SE0015962477,SE0015988019,SE0018012486,SE0018013849
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,27.137428,78.387642,37.431538,40.340775,35.277679,24.287655,46.695545,62.684044,63.488369,7.654094,...,NaN,NaN,NaN,54.399036,6.139913,15.064403,2.877228,3.614271,25.338188,15.991792
2010-01-05,27.194559,78.387642,37.603241,41.408329,35.494995,24.186434,47.185707,63.376675,64.721153,7.944756,...,NaN,NaN,NaN,55.406429,6.235000,15.064403,3.001246,3.729213,25.219095,15.776655
2010-01-07,27.194559,77.850723,36.916424,42.120037,35.784748,24.559349,46.858929,63.376675,65.645737,8.138528,...,NaN,NaN,NaN,58.092800,6.302919,13.773167,2.939237,3.793069,25.219095,16.063503
2010-01-08,27.137428,78.387642,37.774948,42.249443,35.350117,25.438370,47.120354,63.723011,65.645737,8.380747,...,NaN,NaN,NaN,61.618664,6.357254,13.773167,2.976442,3.831384,25.569363,17.856306
2010-01-11,26.794645,77.313835,38.427422,42.831745,34.987919,25.736710,47.381771,63.723011,68.727699,8.913628,...,NaN,NaN,NaN,60.443382,6.438758,13.988370,2.864825,3.831384,25.646420,17.354321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-23,109.130219,485.646912,219.865067,199.625015,147.517456,95.918304,138.419556,317.484070,56.695541,239.486389,...,50.198044,269.631317,49.947170,264.149750,149.288803,147.465515,79.953339,78.615593,75.494644,13.360000
2021-04-26,111.481010,481.082184,220.255051,198.519058,149.866440,97.113266,138.608383,315.923950,56.695541,243.561920,...,49.805870,273.268341,50.789532,265.377472,146.342316,150.414825,81.094170,79.085609,76.041710,13.420000
2021-04-27,110.124786,481.280670,220.742569,199.993668,149.396652,97.572861,137.522552,316.899017,58.853500,247.443390,...,49.511742,275.725800,54.406738,263.205353,148.306656,146.482422,82.615280,79.332985,76.041710,13.080000


,NO0003033102,NO0003049405,NO0003053308,NO0003053605,NO0003078800,NO0003079709,NO0003080608,NO0003095309,NO0003097503,NO0003103103,...,NO0010763550,NO0010791353,NO0010813843,NO0010816093,NO0010816895,NO0010833262,NO0010840507,NO0010861115,NO0010927288,NO0012535832
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,10.013632,1.591135,219.468338,31.237755,63.867905,2.354740,84255.750000,20.642979,15.093127,22.200001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,9.995225,1.588509,224.887329,31.068068,63.572220,2.295708,87037.781250,20.249781,15.093127,22.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-06,10.400188,1.575381,219.468338,31.268608,63.867905,2.315385,85448.046875,20.839579,14.338472,21.400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,10.621077,1.575381,216.758865,33.058033,65.287178,2.295708,87832.632812,20.544680,14.338472,21.299999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,11.688707,1.509740,210.797989,33.212296,64.991493,2.341622,84653.187500,21.724277,14.338472,21.799999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-23,2.770000,11.160000,1.980000,75.669472,109.581871,19.865913,6.800000,5.100000,86.557137,6.440000,...,15.225532,10.640926,1.020,31.937046,12.359623,43.364788,84.199997,37.700001,52.887573,36.072861
2021-04-26,2.810000,11.040000,2.040000,77.316963,113.249886,19.527567,6.660000,4.970000,86.952377,6.400000,...,15.677327,11.007854,0.985,32.194012,12.723141,42.776123,88.500000,37.750000,52.887573,37.898331
2021-04-27,2.772000,11.040000,2.005000,76.780571,110.682274,19.914251,6.690000,4.970000,87.545235,6.400000,...,15.406248,11.582177,0.990,29.202208,12.586823,43.168568,89.349998,37.200001,52.887573,39.088852


In [80]:
datacomb = pd.concat([data_dk, data_se,data_no], sort=False, axis=1, join='inner')
datacomb['Adj Close']

,DK0010219070,DK0010234467,DK0010267129,DK0010272202,DK0010272632,DK0010311471,DK0060079531,DK0060520450,DK0060522746,DK0060542181,...,NO0010763550,NO0010791353,NO0010813843,NO0010816093,NO0010816895,NO0010833262,NO0010840507,NO0010861115,NO0010927288,NO0012535832
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,523.826782,293.015289,5.163384,92.000000,27.892155,84.648483,88.650734,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,534.433960,289.625641,5.163384,89.599998,27.516500,85.582100,88.604576,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,518.115417,283.223022,4.940824,88.000000,27.704327,85.893303,93.267967,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,507.508209,283.147675,5.519478,89.000000,27.704327,87.449341,93.914368,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-11,514.035828,282.469696,5.296919,89.900002,27.516500,87.138138,96.038292,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-23,2500.000000,225.237671,179.800003,2251.000000,556.785645,149.667099,1305.774414,18.820000,122.0,121.800003,...,15.225532,10.640926,1.020,31.937046,12.359623,43.364788,84.199997,37.700001,52.887573,36.072861
2021-04-26,2515.000000,234.089813,178.399994,2271.000000,560.765503,152.655167,1287.446045,19.700001,121.0,122.000000,...,15.677327,11.007854,0.985,32.194012,12.723141,42.776123,88.500000,37.750000,52.887573,37.898331
2021-04-27,2465.000000,234.188171,176.000000,2269.000000,563.949463,152.743057,1374.629761,20.000000,121.0,119.250000,...,15.406248,11.582177,0.990,29.202208,12.586823,43.168568,89.349998,37.200001,52.887573,39.088852


In [81]:
test = datacomb['Adj Close'].ffill().pct_change()
test.index = pd.to_datetime(test.index)
display(test)

#test['Date'] = pd.to_datetime(test['Date'])
#test = test.set_index('date') 
yearly = test.resample('y').sum()
display(yearly)

,DK0010219070,DK0010234467,DK0010267129,DK0010272202,DK0010272632,DK0010311471,DK0060079531,DK0060520450,DK0060522746,DK0060542181,...,NO0010763550,NO0010791353,NO0010813843,NO0010816093,NO0010816895,NO0010833262,NO0010840507,NO0010861115,NO0010927288,NO0012535832
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,0.020249,-0.011568,0.000000,-0.026087,-0.013468,0.011029,-0.000521,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,-0.030534,-0.022107,-0.043104,-0.017857,0.006826,0.003636,0.052631,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,-0.020473,-0.000266,0.117117,0.011364,0.000000,0.018116,0.006931,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-11,0.012862,-0.002394,-0.040322,0.010112,-0.006780,-0.003559,0.022616,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-23,0.002004,0.023692,-0.084521,-0.017460,0.008652,0.009484,0.008031,-0.022845,-0.016129,-0.011364,...,0.005970,0.032508,-0.037736,0.023529,0.003690,0.018433,0.002381,0.060478,-0.018018,-0.009804
2021-04-26,0.006000,0.039301,-0.007786,0.008885,0.007148,0.019965,-0.014036,0.046759,-0.008197,0.001642,...,0.029674,0.034483,-0.034314,0.008046,0.029412,-0.013575,0.051069,0.001326,0.000000,0.050605
2021-04-27,-0.019881,0.000420,-0.013453,-0.000881,0.005678,0.000576,0.067718,0.015228,0.000000,-0.022541,...,-0.017291,0.052174,0.005076,-0.092930,-0.010714,0.009174,0.009605,-0.014570,0.000000,0.031414


,DK0010219070,DK0010234467,DK0010267129,DK0010272202,DK0010272632,DK0010311471,DK0060079531,DK0060520450,DK0060522746,DK0060542181,...,NO0010763550,NO0010791353,NO0010813843,NO0010816093,NO0010816895,NO0010833262,NO0010840507,NO0010861115,NO0010927288,NO0012535832
Date,,,,,,,,,,,,,,,,,,,,,
2010-12-31,0.186490,0.387050,0.898411,-0.075819,0.612079,0.145582,0.296950,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2011-12-31,-0.291666,-0.344397,0.185827,-0.415366,0.002387,-0.453485,-0.129627,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2012-12-31,0.368143,0.050303,0.015282,0.828959,0.567001,0.132289,0.383085,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-12-31,0.458377,-0.031365,1.006213,1.092213,0.525248,0.385727,0.221596,-0.113506,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2014-12-31,-0.281369,-0.038957,0.846780,0.592240,0.044573,0.306970,0.084203,-0.451521,0.097422,-0.011990,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015-12-31,0.373907,-0.033120,0.424047,1.013160,-0.030050,0.225858,0.400164,0.989476,0.392696,0.379846,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2016-12-31,0.289250,0.255649,0.669793,0.325874,0.200150,0.051845,0.191147,0.506477,0.280642,0.023000,...,0.066711,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-12-31,0.350980,0.263961,0.102157,-0.084098,0.344346,0.190585,0.459372,-0.142953,0.183921,0.056319,...,-0.055852,0.162807,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2018-12-31,-0.011885,-0.141833,0.133720,0.129403,0.253479,-0.390992,-0.098637,-1.438139,-0.096928,-0.216370,...,-0.133889,-0.452708,0.109927,-0.150669,0.000000,0.000000,0.000000,0.000000,0.000000,-0.420457


In [82]:
data = datacomb['Adj Close'].ffill().pct_change()
cov_matrix = data.cov()
mean_return = data.mean()*260
display(mean_return)
display(cov_matrix)

DK0010219070    0.200375
DK0010234467    0.044634
DK0010267129    0.428504
DK0010272202    0.397166
DK0010272632    0.325477
                  ...   
NO0010833262    0.162620
NO0010840507    0.150052
NO0010861115    0.222711
NO0010927288    0.131732
NO0012535832    1.276498
Length: 116, dtype: float64

,DK0010219070,DK0010234467,DK0010267129,DK0010272202,DK0010272632,DK0010311471,DK0060079531,DK0060520450,DK0060522746,DK0060542181,...,NO0010763550,NO0010791353,NO0010813843,NO0010816093,NO0010816895,NO0010833262,NO0010840507,NO0010861115,NO0010927288,NO0012535832
DK0010219070,0.000449,0.000159,0.000108,0.000117,0.000123,0.000105,0.000122,0.000055,0.000046,0.000119,...,0.000083,0.000168,0.000013,0.000262,0.000298,0.000243,0.000215,0.000219,-0.000004,0.000206
DK0010234467,0.000159,0.000486,0.000120,0.000139,0.000147,0.000162,0.000164,0.000089,0.000050,0.000164,...,0.000081,0.000188,0.000076,0.000329,0.000403,0.000176,0.000196,0.000248,0.000051,0.000083
DK0010267129,0.000108,0.000120,0.000752,0.000127,0.000096,0.000089,0.000081,0.000095,0.000042,0.000084,...,0.000048,0.000097,-0.000041,0.000117,0.000166,0.000093,0.000121,0.000086,0.000065,0.000198
DK0010272202,0.000117,0.000139,0.000127,0.000803,0.000150,0.000129,0.000146,0.000113,0.000047,0.000087,...,0.000027,0.000063,0.000023,0.000118,0.000110,0.000174,0.000210,0.000087,-0.000028,0.000168
DK0010272632,0.000123,0.000147,0.000096,0.000150,0.000373,0.000101,0.000131,0.000103,0.000045,0.000076,...,0.000059,0.000112,0.000049,0.000172,0.000203,0.000083,0.000219,0.000121,0.000031,0.000222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NO0010833262,0.000243,0.000176,0.000093,0.000174,0.000083,0.000135,0.000161,0.000331,0.000048,0.000069,...,0.000228,0.000355,0.000248,0.000160,0.000134,0.001411,-0.000083,0.000116,-0.000200,0.000268
NO0010840507,0.000215,0.000196,0.000121,0.000210,0.000219,-0.000047,0.000179,0.000214,0.000073,0.000151,...,0.000045,0.000292,-0.000327,0.000142,0.000141,-0.000083,0.001210,0.000154,-0.000013,0.000120
NO0010861115,0.000219,0.000248,0.000086,0.000087,0.000121,0.000151,0.000168,0.000243,0.000089,0.000235,...,0.000163,0.000295,-0.000089,0.000327,0.000225,0.000116,0.000154,0.000783,0.000073,0.000291
NO0010927288,-0.000004,0.000051,0.000065,-0.000028,0.000031,0.000025,0.000018,0.000031,0.000059,0.000009,...,0.000031,-0.000204,-0.000046,0.000053,0.000011,-0.000200,-0.000013,0.000073,0.000417,0.000072


In [83]:
yf.pdr_override() # <== that's all it takes :-)

# download dataframe
data = pdr.get_data_yahoo(isindata, start="2010-01-01", end="2021-04-30")

[*********************100%***********************]  143 of 143 completed

7 Failed downloads:
- : No timezone found, symbol may be delisted
- BRILL.ST: Data doesn't exist for startDate = 1262300400, endDate = 1619733600
- SFL.ST: Data doesn't exist for startDate = 1262300400, endDate = 1619733600
- SDRL.OL: Data doesn't exist for startDate = 1262300400, endDate = 1619733600
- ELO.OL: Data doesn't exist for startDate = 1262300400, endDate = 1619733600
- PURE.ST: Data doesn't exist for startDate = 1262300400, endDate = 1619733600
- INT.ST: Data doesn't exist for startDate = 1262300400, endDate = 1619733600


In [84]:
data

Adj Close                                                      \
           BMG0702P1086 BMG1738J1247 BMG359472021 BMG3682E1921 BMG396372051   
Date                                                                          
2010-01-04          NaN    77.019249    56.691559    83.675407    25.678799   
2010-01-05          NaN    78.234901    56.691559    88.356339    25.678799   
2010-01-06          NaN    76.411415    56.691559    88.443565    25.697680   
2010-01-07          NaN    76.932426    59.053703    91.583588    26.471243   
2010-01-08          NaN    76.932426    58.581276    93.531540    28.225935   
...                 ...          ...          ...          ...          ...   
2021-04-23    26.799999    31.527788    92.545624     6.877536     5.572651   
2021-04-26    27.250000    32.220921    96.952568     6.831066     5.674219   
2021-04-27    26.600000    32.181320    97.246361     6.719538     5.769015   
2021-04-28    26.700001    32.676411    95.287735     6.951888     5.931523   
2021-04-29    28.299999    33.310143    94.749100     7.156354     5.917980   

                                                                             \
           BMG671801022 BMG7997W1029 CA31730E1016 CA5503721063 CA98936C1068   
Date                                                                          
2010-01-04          NaN          NaN          NaN     3.771499          NaN   
2010-01-05          NaN          NaN          NaN     3.914935          NaN   
2010-01-06          NaN          NaN          NaN     3.999307          NaN   
2010-01-07          NaN          NaN          NaN     4.066808          NaN   
2010-01-08          NaN          NaN          NaN     4.125868          NaN   
...                 ...          ...          ...          ...          ...   
2021-04-23    19.430000          NaN         3.20    13.556196        0.925   
2021-04-26    19.740000          NaN         3.28    13.985246        0.925   
2021-04-27    19.870001          NaN         3.41    14.323010        0.925   
2021-04-28    20.280001          NaN         3.32    14.423427        0.900   
2021-04-29    20.379999          NaN         3.34    13.893958        0.875   

            ...       Volume                                         \
            ... SE0015811625 SE0015962477 SE0015988019 SE0016075063   
Date        ...                                                       
2010-01-04  ...          NaN     360940.0    1127724.0          NaN   
2010-01-05  ...          NaN      67830.0    1723959.0          NaN   
2010-01-06  ...          NaN          NaN          NaN          NaN   
2010-01-07  ...          NaN      55240.0    2633823.0          NaN   
2010-01-08  ...          NaN      64800.0    1786021.0          NaN   
...         ...          ...          ...          ...          ...   
2021-04-23  ...          NaN     171780.0    1943984.0          NaN   
2021-04-26  ...          NaN     186280.0    1282940.0          NaN   
2021-04-27  ...          NaN     258710.0    2408136.0          NaN   
2021-04-28  ...          NaN     235150.0    2143136.0          NaN   
2021-04-29  ...          NaN     174020.0    2647364.0          NaN   

                                                                             \
           SE0017232838 SE0018012486 SE0018013849 SGXZ80461361 US36467X2062   
Date                                                                          
2010-01-04          NaN        578.0     155930.0          NaN          0.0   
2010-01-05          NaN        453.0      88278.0          NaN          0.0   
2010-01-06          NaN          NaN          NaN          NaN          0.0   
2010-01-07          NaN        653.0     109843.0          NaN          0.0   
2010-01-08          NaN        423.0     656391.0          NaN          0.0   
...                 ...          ...          ...          ...          ...   
2021-04-23          NaN        303.0     102731.0          0.0     130870.0   
2021-04-26     

In [85]:
datatest = data['Adj Close']
datatest

,BMG0702P1086,BMG1738J1247,BMG359472021,BMG3682E1921,BMG396372051,BMG671801022,BMG7997W1029,CA31730E1016,CA5503721063,CA98936C1068,...,SE0015811625,SE0015962477,SE0015988019,SE0016075063,SE0017232838,SE0018012486,SE0018013849,SGXZ80461361,US36467X2062,US7170811035
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,77.019249,56.691559,83.675407,25.678799,NaN,NaN,NaN,3.771499,NaN,...,NaN,2.877227,3.614271,NaN,NaN,25.338186,15.991792,NaN,30.600000,11.026820
2010-01-05,NaN,78.234901,56.691559,88.356339,25.678799,NaN,NaN,NaN,3.914935,NaN,...,NaN,3.001245,3.729212,NaN,NaN,25.219099,15.776655,NaN,30.600000,10.869542
2010-01-06,NaN,76.411415,56.691559,88.443565,25.697680,NaN,NaN,NaN,3.999307,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.600000,10.834592
2010-01-07,NaN,76.932426,59.053703,91.583588,26.471243,NaN,NaN,NaN,4.066808,NaN,...,NaN,2.939237,3.793069,NaN,NaN,25.219099,16.063503,NaN,30.600000,10.793819
2010-01-08,NaN,76.932426,58.581276,93.531540,28.225935,NaN,NaN,NaN,4.125868,NaN,...,NaN,2.976442,3.831383,NaN,NaN,25.569363,17.856306,NaN,30.600000,10.881191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-23,26.799999,31.527788,92.545624,6.877536,5.572651,19.430000,NaN,3.20,13.556196,0.925,...,NaN,79.953331,78.615593,NaN,NaN,75.494644,13.360000,15.849358,19.780001,36.094685
2021-04-26,27.250000,32.220921,96.952568,6.831066,5.674219,19.740000,NaN,3.28,13.985246,0.925,...,NaN,81.094170,79.085617,NaN,NaN,76.041710,13.420000,15.849358,20.750000,36.113358
2021-04-27,26.600000,32.181320,97.246361,6.719538,5.769015,19.870001,NaN,3.41,14.323010,0.925,...,NaN,82.615280,79.332993,NaN,NaN,76.041710,13.080000,15.849358,20.799999,35.898621


In [86]:
result = []
for col in datatest.columns:
    #print(col)
    print(datatest[col].isnull().all())
    if datatest[col].isnull().all() == True:
        result.append(col)
    

False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
False
False
True
True
False
False
False
False
False


In [87]:
result

['BMG7997W1029',
 'DK0061277894',
 'NO0011002586',
 'SE0015346895',
 'SE0015811625',
 'SE0016075063',
 'SE0017232838']

In [88]:
datatest = datatest.drop(columns=result)
datatest

,BMG0702P1086,BMG1738J1247,BMG359472021,BMG3682E1921,BMG396372051,BMG671801022,CA31730E1016,CA5503721063,CA98936C1068,DK0010219070,...,SE0014504817,SE0014781795,SE0014960365,SE0015962477,SE0015988019,SE0018012486,SE0018013849,SGXZ80461361,US36467X2062,US7170811035
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,77.019249,56.691559,83.675407,25.678799,NaN,NaN,3.771499,NaN,523.826904,...,54.399036,6.139912,15.064399,2.877227,3.614271,25.338186,15.991792,NaN,30.600000,11.026820
2010-01-05,NaN,78.234901,56.691559,88.356339,25.678799,NaN,NaN,3.914935,NaN,534.433960,...,55.406425,6.234999,15.064399,3.001245,3.729212,25.219099,15.776655,NaN,30.600000,10.869542
2010-01-06,NaN,76.411415,56.691559,88.443565,25.697680,NaN,NaN,3.999307,NaN,514.035706,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.600000,10.834592
2010-01-07,NaN,76.932426,59.053703,91.583588,26.471243,NaN,NaN,4.066808,NaN,518.115356,...,58.092804,6.302920,13.773166,2.939237,3.793069,25.219099,16.063503,NaN,30.600000,10.793819
2010-01-08,NaN,76.932426,58.581276,93.531540,28.225935,NaN,NaN,4.125868,NaN,507.508209,...,61.618675,6.357255,13.773166,2.976442,3.831383,25.569363,17.856306,NaN,30.600000,10.881191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-23,26.799999,31.527788,92.545624,6.877536,5.572651,19.430000,3.20,13.556196,0.925,2500.000000,...,264.149750,149.288803,147.465515,79.953331,78.615593,75.494644,13.360000,15.849358,19.780001,36.094685
2021-04-26,27.250000,32.220921,96.952568,6.831066,5.674219,19.740000,3.28,13.985246,0.925,2515.000000,...,265.377472,146.342316,150.414825,81.094170,79.085617,76.041710,13.420000,15.849358,20.750000,36.113358
2021-04-27,26.600000,32.181320,97.246361,6.719538,5.769015,19.870001,3.41,14.323010,0.925,2465.000000,...,263.205353,148.306656,146.482422,82.615280,79.332993,76.041710,13.080000,15.849358,20.799999,35.898621


In [89]:
datatest = datatest.ffill().pct_change()
datatest

,BMG0702P1086,BMG1738J1247,BMG359472021,BMG3682E1921,BMG396372051,BMG671801022,CA31730E1016,CA5503721063,CA98936C1068,DK0010219070,...,SE0014504817,SE0014781795,SE0014960365,SE0015962477,SE0015988019,SE0018012486,SE0018013849,SGXZ80461361,US36467X2062,US7170811035
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,NaN,0.015784,0.000000,0.055942,0.000000,NaN,NaN,0.038031,NaN,0.020249,...,0.018519,0.015487,0.000000,0.043103,0.031802,-0.004700,-0.013453,NaN,0.000000,-0.014263
2010-01-06,NaN,-0.023308,0.000000,0.000987,0.000735,NaN,NaN,0.021551,NaN,-0.038168,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,-0.003215
2010-01-07,NaN,0.006819,0.041667,0.035503,0.030102,NaN,NaN,0.016878,NaN,0.007937,...,0.048485,0.010893,-0.085714,-0.020661,0.017123,0.000000,0.018182,NaN,0.000000,-0.003763
2010-01-08,NaN,0.000000,-0.008000,0.021270,0.066287,NaN,NaN,0.014523,NaN,-0.020473,...,0.060694,0.008621,0.000000,0.012658,0.010101,0.013889,0.111607,NaN,0.000000,0.008095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-23,-0.001862,0.003783,-0.004739,0.006803,0.031328,0.007258,0.015873,0.004057,0.000000,0.002004,...,-0.006041,0.013333,0.001335,-0.008255,0.009530,0.053435,0.004511,0.0,-0.020792,0.000518
2021-04-26,0.016791,0.021985,0.047619,-0.006757,0.018226,0.015955,0.025000,0.031650,0.000000,0.006000,...,0.004648,-0.019737,0.020000,0.014269,0.005979,0.007246,0.004491,0.0,0.049039,0.000517
2021-04-27,-0.023853,-0.001229,0.003030,-0.016327,0.016706,0.006586,0.039634,0.024152,0.000000,-0.019881,...,-0.008185,0.013423,-0.026144,0.018757,0.003128,0.000000,-0.025335,0.0,0.002410,-0.005946


In [90]:
datatest = datatest[1:]
datatest

,BMG0702P1086,BMG1738J1247,BMG359472021,BMG3682E1921,BMG396372051,BMG671801022,CA31730E1016,CA5503721063,CA98936C1068,DK0010219070,...,SE0014504817,SE0014781795,SE0014960365,SE0015962477,SE0015988019,SE0018012486,SE0018013849,SGXZ80461361,US36467X2062,US7170811035
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-05,NaN,0.015784,0.000000,0.055942,0.000000,NaN,NaN,0.038031,NaN,0.020249,...,0.018519,0.015487,0.000000,0.043103,0.031802,-0.004700,-0.013453,NaN,0.000000,-0.014263
2010-01-06,NaN,-0.023308,0.000000,0.000987,0.000735,NaN,NaN,0.021551,NaN,-0.038168,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,-0.003215
2010-01-07,NaN,0.006819,0.041667,0.035503,0.030102,NaN,NaN,0.016878,NaN,0.007937,...,0.048485,0.010893,-0.085714,-0.020661,0.017123,0.000000,0.018182,NaN,0.000000,-0.003763
2010-01-08,NaN,0.000000,-0.008000,0.021270,0.066287,NaN,NaN,0.014523,NaN,-0.020473,...,0.060694,0.008621,0.000000,0.012658,0.010101,0.013889,0.111607,NaN,0.000000,0.008095
2010-01-11,NaN,-0.002257,0.080645,0.057818,0.047460,NaN,NaN,0.044990,NaN,0.012862,...,-0.019074,0.012821,0.015625,-0.037500,0.000000,0.003014,-0.028113,NaN,0.000000,0.008030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-23,-0.001862,0.003783,-0.004739,0.006803,0.031328,0.007258,0.015873,0.004057,0.000000,0.002004,...,-0.006041,0.013333,0.001335,-0.008255,0.009530,0.053435,0.004511,0.0,-0.020792,0.000518
2021-04-26,0.016791,0.021985,0.047619,-0.006757,0.018226,0.015955,0.025000,0.031650,0.000000,0.006000,...,0.004648,-0.019737,0.020000,0.014269,0.005979,0.007246,0.004491,0.0,0.049039,0.000517
2021-04-27,-0.023853,-0.001229,0.003030,-0.016327,0.016706,0.006586,0.039634,0.024152,0.000000,-0.019881,...,-0.008185,0.013423,-0.026144,0.018757,0.003128,0.000000,-0.025335,0.0,0.002410,-0.005946


In [91]:
datatest.cov()

,BMG0702P1086,BMG1738J1247,BMG359472021,BMG3682E1921,BMG396372051,BMG671801022,CA31730E1016,CA5503721063,CA98936C1068,DK0010219070,...,SE0014504817,SE0014781795,SE0014960365,SE0015962477,SE0015988019,SE0018012486,SE0018013849,SGXZ80461361,US36467X2062,US7170811035
BMG0702P1086,0.002274,0.001613,0.000545,0.000086,0.000671,0.001193,0.000292,0.000638,0.000533,0.000549,...,0.000662,0.000436,0.000393,0.000542,0.000401,0.000277,0.000363,0.000043,0.000317,0.000303
BMG1738J1247,0.001613,0.001314,0.000156,0.000211,0.000272,0.000428,0.000125,0.000237,0.000224,0.000165,...,0.000166,0.000131,0.000083,0.000159,0.000163,0.000115,0.000117,0.000006,0.000101,0.000056
BMG359472021,0.000545,0.000156,0.002003,0.000127,0.000116,0.000148,0.000043,0.000088,0.000038,0.000057,...,0.000078,0.000052,-0.000004,0.000054,0.000060,0.000024,0.000049,0.000004,0.000081,0.000033
BMG3682E1921,0.000086,0.000211,0.000127,0.001767,0.000498,0.000108,0.000029,0.000317,0.000070,0.000085,...,0.000109,0.000095,0.000050,0.000087,0.000127,0.000123,0.000054,-0.000025,0.000178,0.000114
BMG396372051,0.000671,0.000272,0.000116,0.000498,0.001281,0.000197,0.000141,0.000359,0.000096,0.000120,...,0.000103,0.000108,0.000084,0.000110,0.000125,0.000101,0.000051,0.000038,0.000051,0.000107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SE0018012486,0.000277,0.000115,0.000024,0.000123,0.000101,0.000089,0.000029,0.000110,0.000029,0.000089,...,0.000068,0.000076,0.000059,0.000087,0.000113,0.000798,0.000113,0.000019,0.000062,0.000037
SE0018013849,0.000363,0.000117,0.000049,0.000054,0.000051,0.000096,0.000015,0.000135,0.000232,0.000066,...,0.000044,0.000052,0.000026,0.000074,0.000092,0.000113,0.002429,-0.000002,0.000071,0.000035
SGXZ80461361,0.000043,0.000006,0.000004,-0.000025,0.000038,0.000029,0.000032,0.000031,-0.000034,0.000033,...,-0.000007,0.000012,0.000019,0.000010,0.000016,0.000019,-0.000002,0.000881,-0.000004,0.000012
US36467X2062,0.000317,0.000101,0.000081,0.000178,0.000051,0.000066,0.000100,0.000088,0.000069,0.000081,...,0.000030,0.000050,0.000050,0.000055,0.000063,0.000062,0.000071,-0.000004,0.002615,0.000022


In [92]:
meanreturns = datatest.mean()

In [93]:
meanreturns_year = meanreturns*260
meanreturns_year

BMG0702P1086    0.442553
BMG1738J1247    0.094777
BMG359472021    0.281814
BMG3682E1921    0.010133
BMG396372051    0.035024
                  ...   
SE0018012486    0.196113
SE0018013849    0.279838
SGXZ80461361    0.161849
US36467X2062    0.289113
US7170811035    0.126188
Length: 136, dtype: float64

In [94]:
from scipy.optimize import minimize

In [95]:
def portfolio_performance(returns_mean,cov_matrix):
    
    n = np.shape(cov_matrix)[0]
    
    #Define a bunch of lists that are used later
    weight = np.zeros((num_portfolios,n))
    expectedReturn = np.zeros(num_portfolios)
    expectedVolatility = np.zeros(num_portfolios)
    SharpeRatio = np.zeros(num_portfolios)
    
    #We use a minimizer so we create a helper function find the negative sharpe ratio, which will be the highest
    def negativeSR(w):
        w = np.array(w)
        V = np.sqrt(w.T @ cov_matrix @ w)
        R = np.sum(returns_mean * w)
        SR = (R-rf)/V
        return -1*SR

    #Helper function used as a constraint
    def checkSumToOne(w):
        return np.sum(w)-1

    #Helper function to find the expected return of a portfolio given certain weights
    def getReturn(w):
        w = np.array(w)
        R = np.sum(returns_mean * w)
        return R    
    
    
    #Define the linspace for the risk free rates
    lin = np.linspace(0.20,0.00)
    
    #Create a list for optimal volatility weights
    volatility_opt = []
    
    #Create the initial guess
    w0 = n*[1./n]
    
    #Create the constraints
    constraintSet = (0,1)
    bounds = tuple(constraintSet for asset in range(n))
    
    #Loop through each target value and find the optimal volatility
    for rf in lin:

        constraints = ({'type':'eq', 'fun':checkSumToOne})
        w_opt = minimize(negativeSR, w0, method='SLSQP', bounds=bounds, constraints=constraints).x
        volatility_opt.append(opt['fun'])
    
    return expectedReturn, expectedVolatility, SharpeRatio, volatility_opt, (sharpe_exp,sharpe_vol,(sharpe_exp-riskfree_rate) / sharpe_vol), lin, w_opt

In [96]:
NPmeanreturns_year = meanreturns_year.to_numpy()
NPcov = datatest.cov().to_numpy()

In [97]:
np.shape(NPcov)[0]

136

In [98]:
returns_mean = NPmeanreturns_year
cov_matrix = NPcov
def negativeSR(w):
    w = np.array(w)
    V = np.sqrt(w.T @ cov_matrix @ w)
    R = np.sum(returns_mean * w)
    SR = (R-rf)/V
    return -1*SR

def checkSumToOne(w):
    return np.sum(w)-1

In [99]:
lin = np.linspace(-0.20,0.20)

n = np.shape(cov_matrix)[0]
    
#Create a list for optimal volatility weights
volatility_opt = []
    
#Create the initial guess
w0 = n*[1./n]
    
#Create the constraints
constraintSet = (0,1)
bounds = tuple(constraintSet for asset in range(n))
    
#Loop through each target value and find the optimal volatility
for rf in lin:

    constraints = ({'type':'eq', 'fun':checkSumToOne})
    w_opt = minimize(negativeSR, w0, method='SLSQP', bounds=bounds, constraints=constraints).x
    volatility_opt.append(w_opt)

KeyboardInterrupt: 

In [ ]:
volatility_opt

In [ ]:
print(len(w_opt))

In [ ]:
np.sum(returns_mean * w_opt)

In [ ]:
np.sqrt(w_opt.T @ cov_matrix @ w_opt)

In [ ]:
(np.sum(returns_mean * w_opt)-rf)/np.sqrt(w_opt.T @ cov_matrix @ w_opt)

In [ ]:
np.shape(volatility_opt)

In [ ]:
portfolioreturns = []
portfoliostds = []
for i in range(50):
    
    print(i)
    
    w_opt = volatility_opt[i]
    
    portfolioreturn = np.sum(returns_mean * w_opt)
    portfoliostd = np.sqrt(w_opt.T @ cov_matrix @ w_opt)
    
    portfolioreturns.append(portfolioreturn)
    portfoliostds.append(portfoliostd)

In [ ]:
plt.scatter(portfoliostds,portfolioreturns)